# Data Analysis (Regression): Train-Test Split

## Import Libraries

In [1]:
# Import Required Modules and Packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import sys

import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
cd

/root


## Load Data

In [3]:
# Upload the X and y Data
final_data_x = pd.read_csv('Project/Cleaned/final_data_x.csv', na_values = ['..'])
final_data_y_continuous = pd.read_csv('Project/Cleaned/final_data_y_continuous.csv', na_values = ['..'])

# Drop Column
final_data_x.drop(['Unnamed: 0'], axis=1, inplace=True)
final_data_y_continuous.drop(['Unnamed: 0'], axis=1, inplace=True)

# Set Indices
final_data_x.set_index(['country', 'year'], inplace=True)

# Load Dictionary
dictionary_series = np.load('Project/Cleaned/dictionary_series.npy',allow_pickle='TRUE').item()

In [4]:
# Shape of Data
print("Shape of Data (Features): ", final_data_x.shape)
print("Shape of Data (Outcome - Continuous): ", final_data_y_continuous.shape)

Shape of Data (Features):  (4992, 714)
Shape of Data (Outcome - Continuous):  (4992, 1)


## Outcome Variables - Continuous
For the purposes of this project, supervised machine learning regression models will be used to predict the continuous value <continuous_conflict_instances> with the value being the number of conflict events of any kind occurring.

## Scale Data

In [5]:
# Scale Data
names =  final_data_x.columns
feature_scaler = StandardScaler()
final_data_x_scaled = feature_scaler.fit_transform(final_data_x)
final_data_x_scaled = pd.DataFrame(final_data_x_scaled, columns=names)

## Feature Selection
### ANOVA F-Value 
ANOVA (Analysis of Variance) checks the means of two or more groups that are significantly different from each other to select which features to include in the model.

In [6]:
# Define feature selection
featureselection = SelectKBest(score_func=f_regression, k=20)
X_selectedfeatures = featureselection.fit_transform(final_data_x_scaled, final_data_y_continuous.values.ravel())

print('Shape of X_selectedfeatures:       ', X_selectedfeatures.shape)
print('Selected Features (ANOVA F-Value): ', X_selectedfeatures.shape[1])

Shape of X_selectedfeatures:        (4992, 20)
Selected Features (ANOVA F-Value):  20


In [7]:
# Feature Columns
feature_columns = final_data_x_scaled.columns

selected_features = pd.DataFrame(featureselection.inverse_transform(X_selectedfeatures), 
                                 index=final_data_x_scaled.index, 
                                 columns=feature_columns)

# Selected Feature Columns
selected_columns = selected_features.columns[selected_features.var() != 0]
selected_columns

Index(['DT.NFL.UNDP.CD', 'DT.NFL.UNFP.CD', 'EG.CFT.ACCS.ZS',
       'EG.ELC.ACCS.RU.ZS', 'EG.ELC.ACCS.UR.ZS', 'EG.FEC.RNEW.ZS',
       'IT.MLT.MAIN.P2', 'NV.AGR.TOTL.ZS', 'SE.PRM.ENRL.TC.ZS', 'SE.PRM.NENR',
       'SE.PRM.TCHR.FE.ZS', 'SE.SEC.ENRR', 'SE.SEC.NENR', 'SH.ANM.ALLW.ZS',
       'SH.DTH.COMM.ZS', 'SH.DYN.0514', 'SH.STA.BRTC.ZS', 'SH.TBS.DTEC.ZS',
       'SL.AGR.EMPL.FE.ZS', 'SP.ADO.TFRT'],
      dtype='object')

In [8]:
dictionary_series['DT.NFL.UNDP.CD']

{'Series Name': 'Net official flows from UN agencies, UNDP (current US$)'}

In [9]:
# Filter out Unselected Feature Columns
final_data_x_scaled_filtered = final_data_x_scaled[selected_columns]

In [10]:
# Shape of Data
print('Shape of Data (Selected Feature Columns):', final_data_x_scaled_filtered.shape)

Shape of Data (Selected Feature Columns): (4992, 20)


## Train-Test Data

In [11]:
# Train-Test Split
X = final_data_x_scaled_filtered
y = final_data_y_continuous
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# Shape of Train and Test Data
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)

X_test: (999, 20)
y_test: (999, 1)
X_train: (3993, 20)
y_train: (3993, 1)


## Save Train-Test Data

In [12]:
# Save Data
X.to_csv('Project/Cleaned/Regression/X.csv')
y.to_csv('Project/Cleaned/Regression/y.csv')
X_train.to_csv('Project/Cleaned/Regression/X_train.csv')
X_test.to_csv('Project/Cleaned/Regression/X_test.csv')
y_train.to_csv('Project/Cleaned/Regression/y_train.csv')
y_test.to_csv('Project/Cleaned/Regression/y_test.csv')